In [ ]:
!pip install datasets torch torchvision transformers tensorboard ipdb --quiet 
# colorama

     |████████████████████████████████| 362 kB 24.0 MB/s 
     |████████████████████████████████| 4.4 MB 49.1 MB/s 
     |████████████████████████████████| 140 kB 72.5 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 101 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 45.9 MB/s 
     |████████████████████████████████| 6.6 MB 35.3 MB/s 
     |████████████████████████████████| 144 kB 60.9 MB/s 
     |████████████████████████████████| 271 kB 52.1 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 

In [ ]:
import sys
import os
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
import ipdb
import os
import random
import numpy as np
import pandas as pd
tqdm.pandas()

DRIVE_PATH = "/content/drive/MyDrive/transformers_layers/"
sys.path.append(DRIVE_PATH)

In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
from tqdm.auto import tqdm 
import pandas as pd 
import os 
import sys 
import torch 
import shutil

torch.cuda.empty_cache() 
import shared.utils as utils 
import shared.constants as constants
utils.set_all_seeds ()
from train_baseline.trainer_models import SuperGlueTrainerBaseline 
from train.trainer_models import SuperGlueTrainer 
import pretrain.trainer_pretrain

print(f"{constants.SAVE_RESULTS}/test.pkl") 
tqdm.pandas()

# %%
def pretrain_and_train(_task, _results, _w_mlm, _w_layers, _w_distance, _w_fake, _w_real):
    torch. cuda. empty_cache()
    pretrain.trainer_pretrain.PretrainTrainer(_task, _w_mlm, _w_layers, _w_distance, _w_fake, _w_real).train()
    lm_path = constants. SAVE_LM_PATH + _task
    print(constants.PRINT_COLOR, "*" * 25 + f" {_task} - AFTER NEW TRAIN OBJ - ALL DATA " + "*" * 25,
    constants.PRINT_END_COLOR) 
    accuracy = SuperGlueTrainer(_task, _lm_path=lm_path, _data_limit=None).train()
    _results.append(dict(task=_task, is_train_obj=True, is_limited_data=False, accuracy=accuracy, _w_mlm=_w_mlm,
    _w_layers=_w_layers,
    _w_distance=_w_distance, _w_fake=_w_fake, _w_real=_w_real))
    print(constants.PRINT_COLOR, "*" * 25 + F" {_task} - AFTER NEW TRAIN OBJ - LIMITED DATA " + "*" * 25,
    constants.PRINT_END_COLOR) 
    accuracy = SuperGlueTrainer(_task, _lm_path=lm_path,
    mit=constants.DATA_LIMIT).train() 
    _results.append(
        dict(
            task=_task, 
            is_train_obj=True, 
            is_limited_data=True, 
            accuracy=accuracy, 
            w_mlm=_w_mlm, 
            w_layers=_w_layers,
            w_distance=_w_distance, 
            w_fake=_w_fake, 
            w_real=_w_real
            ))

def baseline(_task, _results, _is_limited_data, _data_limit, _print_title):
    torch.cuda.empty_cache() 
    utils.set_all_seeds()
    print(constants.PRINT_COLOR, "*" * 25 + f" BASELINE - {_task} - {_print_title} " + "*" * 25,
    constants.PRINT_END_COLOR) 
    accuracy = SuperGlueTrainerBaseline(_task, _lm_path=None, _data_limit=_data_limit).train() 
    _results.append(dict(
    task=_task, is_train_obj=False, is_limited_data=_is_limited_data, 
    accuracy=accuracy, is_clip_grad_norm=-1, seq_embed_type=-1, layer_mean_axis=-1, 
    matrix_dist_func=-1, w_m1m=-1, w_layers=-1, w_distance=-1, w_fake=-1, w_real=-1))

def y_train(_task, _results, _print_title, _is_limited_data, _data_limit, _is_clip_grad_norm, _seq_embed_type,
    _layer_mean_axis, _matrix_dist_func, _weights): 
    _w_mlm, _w_layers, _w_distance, _w_fake, _w_real = _weights 
    torch.cuda.empty_cache() 
    utils.set_all_seeds()
    print(constants.PRINT_COLOR, "*" * 25 + f" {_task} - {_print_title} " + "*" * 25, constants.PRINT_END_COLOR) 
    accuracy = SuperGlueTrainer(_task, _lm_path=None, _data_limit=_data_limit, 
    _is_clip_grad_norm=_is_clip_grad_norm, _seq_embed_type=_seq_embed_type, 
    _layer_mean_axis=_layer_mean_axis, _matrix_dist_func=_matrix_dist_func, _w_mlm=_w_mlm,
    _w_layers=_w_layers, _w_distance=_w_distance, _w_fake=_w_fake, _w_real=_w_real).train() 
    _results.append(dict(
        task=_task, 
        is_train_obj=False, 
        is_limited_data=_is_limited_data, 
        accuracy=accuracy, 
        is_clip_grad_norm=_is_clip_grad_norm, 
        seq_embed_type=_seq_embed_type, 
        layer_mean_axis=_layer_mean_axis, 
        matrix_dist_func=_matrix_dist_func, 
        w_mlm=_w_mlm, 
        w_layers=_w_layers, 
        w_distance=_w_distance, 
        w_fake=_w_fake, 
        w_real=_w_real
    ))

# %%

def set_i_weights(_idx, _default_idx):
    w = [_default_idx] * 5 
    w[_idx] = 0 
    return w

def cal_weights():
    w_1 = [set_i_weights(i, _default_idx=1) for i in range(5)] 
    weights_1 = list(map(tuple, w_1)) 
    
    w_2 = [set_i_weights(i, _default_idx=0.4) for i in range(5)] 
    weights_2 = list(map(tuple, w_2)) 
    
    weights_1.extend(weights_2) 
    return weights_1

def run_y_train(_task, _results, _is_clip_grad_norm, _seq_embed_type, _layer_mean_axis, _matrix_dist_func, _weights): 
    y_train(_task, _results, _print_title="REGULAR - ALL DATA", _is_limited_data=False, 
    _data_limit=None, _is_clip_grad_norm=_is_clip_grad_norm, _seq_embed_type=_seq_embed_type, 
    _layer_mean_axis=_layer_mean_axis, _matrix_dist_func=_matrix_dist_func,
    _weights=_weights) 

    # y_train(_task, _results, _print_title="LIMITED - ALL DATA", _is_limited_data=True, _data_limit=constants.DATA_LIMIT, _is_clip_grad_norm=_is_clip_grad_norm, 
    # _seq_embed_type=_seq_embed_type, _layer_mean_axis=_layer_mean_axis, _matrix_dist_func=_matrix_dist_func, 
    # _weights=_weights)


def run_all(_name):
    tasks = [constants.COPA, constants.RTE, constants.CB] 
    results = [] 
    # weights = cal_weights()
    # w_limits = [(7,) * 5, (0,) * 5, (1, 0, 0, 1, 1), (1, 1, 1, 0, 0), (0.4, 0, 0, 0.4, 0.4), (0.4, 0.4, 0.4, 0, 0)]
    # weights.extend(w_limits) 
    weights = [(0,) * 5, (0.2, 0, 0, 0.2, 0.2), (0.2, 0.2, 0.2, 0, 0)]
    utils.set_all_seeds()
    dist_losses = [constants.MATRIX_DIST_ORTHOGONAL, constants.MATRIX_DIST_MUL, constants.MATRIX_DIST_SUM,
    constants.MATRIX_DIST, constants.DIALOG_VECTORS] 
    for task in tasks:
        baseline(task, results, _is_limited_data=False, _data_limit=None, _print_title="REGULAR - ALL DATA")
        baseline(task, results, _is_limited_data=True, _data_limit=constants.DATA_LIMIT,
        _print_title="REGULAR - LIMITED DATA") 
        for w in weights: 
            for dist_loss in dist_losses: 
                for norm_type in [True, False]: 
                    for embed_type in [constants.CLS, constants.MEAN]: 
                        for mean_axis in [1, 2]:
                            run_y_train(task, results, _is_clip_grad_norm=norm_type, _seq_embed_type=embed_type,
                                    _layer_mean_axis=mean_axis, _matrix_dist_func=dist_loss, _weights=w)
                            

        # for w in [(0.5, 0.5, 0.5, 0.5, 0.5)]: 
        #     for dist_loss in dist_losses: 
        #         for norm_type in [True]: 
        #             for embed_type in [constants.CLS]: 
        #                 for mean_axis in [2]:
        #                     run_y_train(task, results, _is_clip_grad_norm=norm_type, _seq_embed_type=embed_type,
        #                             _layer_mean_axis=mean_axis, _matrix_dist_func=dist_loss, _weights=w)

    df = pd.DataFrame(results) 
    df.to_pickle(f"{constants.SAVE_RESULTS}/results_{_name}.pkl")

/content/tests/NEW3/test.pkl


In [ ]:
constants.LM_DEFAULT = constants.ROBERTA_BASE 
run_all("base")

# constants.LM_DEFAULT = constants.ROBERTA_LARGE 
# run_all("large")

 ************************* BASELINE - copa - REGULAR - ALL DATA ************************* 


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

n_layers: 14
**********************************************************************************************************************************

lm path: None
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 46.43% 
 copa - accuracy: 51.79% 
 copa - accuracy: 58.93% 
 copa - accuracy: 48.21% 
 copa - accuracy: 50.00% 
 copa - accuracy: 44.64% 
 copa - accuracy: 53.57% 
 copa - accuracy: 48.21% 
 copa - accuracy: 53.57% 
**********************************************************************************************************************************

 ************************* copa - REGULAR - ALL DATA ************************* 


Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/copa/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


  0%|          | 0/3 [00:00<?, ?it/s]

n_layers: 14
**********************************************************************************************************************************

lm path: None
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 
 copa - accuracy: 50.00% 


In [ ]:
# !zip -r all.zip /content/tests/
# !unzip results.zip
# !cp -r content/tests/NEW2 results

In [ ]:
import pandas as pd
df = pd.read_pickle("content/tests/results_base.pkl")
df = df.sort_values(by=["task", "accuracy"], ascending=False)

In [ ]:
!zip -r all.zip /content/tests/